In [1]:
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo

import alpaca
from alpaca.data.live.stock import *
from alpaca.data.historical.stock import *
from alpaca.data.historical.crypto import *
from alpaca.data.requests import *
from alpaca.data.timeframe import *
from alpaca.trading.client import *
from alpaca.trading.stream import *
from alpaca.trading.requests import *
from alpaca.trading.enums import *
from alpaca.common.exceptions import APIError
import yaml

import yfinance as yf
import pandas as pd
import os


In [2]:
with open("../infra/config.yaml", "r") as file:
    config = yaml.safe_load(file)

TRADE_API_KEY = config["alpaca"]["key"]
TRADE_API_SECRET = config["alpaca"]["secret"]

#### We use paper environment for this example ####
PAPER=True # Please do not modify this. This example is for paper trading only.
####

# Below are the variables for development this documents
# Please do not change these variables
TRADE_API_URL=None
TRADE_API_WSS=None
DATA_API_URL=None
OPTION_STREAM_DATA_WSS=None

api_key = TRADE_API_KEY
secret_key = TRADE_API_SECRET
paper = PAPER
trade_api_url = TRADE_API_URL

stock_historical_data_client = StockHistoricalDataClient(api_key, secret_key, url_override=DATA_API_URL)
crypto_historical_data_client = CryptoHistoricalDataClient(api_key, secret_key, url_override=DATA_API_URL)

In [3]:
def download_data(ticker, start, end = None, interval_type = TimeFrameUnit.Minute, interval_length = 1, crypto = False):
    if crypto:
        req = CryptoBarsRequest(
        symbol_or_symbols=ticker,
        timeframe=TimeFrame(amount=interval_length, unit=interval_type), # specify timeframe
        start=start,                          # specify start datetime, default=the beginning of the current day.
        # end_date=None,                                        # specify end datetime, default=now
        #limit=None,                                               # specify limit
        )
    
        hist = crypto_historical_data_client.get_crypto_bars(req).df
        hist = hist.reset_index().set_index('timestamp')

    else:
        req = StockBarsRequest(
        symbol_or_symbols=ticker,
        timeframe=TimeFrame(amount=interval_length, unit=interval_type), # specify timeframe
        start=start,                          # specify start datetime, default=the beginning of the current day.
        # end_date=None,                                        # specify end datetime, default=now
        #limit=None,                                               # specify limit
        )
    
        hist = stock_historical_data_client.get_stock_bars(req).df
        hist = hist.reset_index().set_index('timestamp')

    return hist

In [5]:
crypto = False

ticker = 'T'
interval_type = TimeFrameUnit.Minute

#START AND END TIME
########################
length = 1825
start_time = datetime.now() - timedelta(days=length)
# end

csv_name = f"{ticker} - {length} Day".replace("/", "_")

download_data(ticker, start_time, end = None, interval_type = TimeFrameUnit.Minute, interval_length = 1, crypto = crypto).to_csv(os.path.join('hist_data', f"{csv_name}.csv"))
